In [ ]:
# Install and import necessary libraries
!pip install gradio
import gradio as gr
import string
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import os

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('punkt')
# Function to clean and process the text input
def clean_text(input_text):
    lower_case = input_text.lower()
    cleaned_text = lower_case.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text into words
    tokenized_words = word_tokenize(cleaned_text, "english")

    # Check if stopwords are available
    try:
        stop_words = set(stopwords.words('english'))
    except LookupError:
        nltk.download('stopwords')
        stop_words = set(stopwords.words('english'))

    # Removing Stop Words
    final_words = [word for word in tokenized_words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(word) for word in final_words]

    return ' '.join(lemma_words)

# Function to perform sentiment analysis
def sentiment_analyze(sentiment_text):
    sia = SentimentIntensityAnalyzer()

    # Check if the lexicon file exists
    if os.path.exists("lexiconalsam.txt"):
        with open("lexiconalsam.txt", 'r') as file:
            for line in file:
                parts = line.strip().split(":")
                if len(parts) == 2:
                    word, score = parts
                    try:
                        sia.lexicon[word] = float(score)
                    except ValueError:
                        pass  # Ignore invalid scores

    score = sia.polarity_scores(sentiment_text)

    # Determine sentiment type
    if score['neg'] > score['pos'] and score['neg'] > score['neu'] :
        sentiment = "Negative Sentiment"
    elif score['neg'] < score['pos']:
        sentiment = "Positive Sentiment"
    else:
        sentiment = "Neutral Sentiment"

    return sentiment, score

# Function to generate pie chart based on sentiment analysis scores
def generate_pie_chart(sentiment_score):
    labels = ['Negative', 'Neutral', 'Positive']
    sizes = [sentiment_score['neg'] * 100, sentiment_score['neu'] * 100, sentiment_score['pos'] * 100]
    colors = ['#ff6666', '#ffcc99', '#66b3ff']

    # Create the pie chart
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    return fig

# Create Gradio interface for sentiment analysis
def analyze_sentiment(enter_your_comment):
    if not enter_your_comment.strip():
        return "No input provided.", {}, None

    cleaned_text = clean_text(enter_your_comment)
    sentiment, score = sentiment_analyze(cleaned_text)

    # Generate the pie chart
    pie_chart = generate_pie_chart(score)

    return sentiment, score, pie_chart

# Create the Gradio interface with input and output
iface = gr.Interface(fn=analyze_sentiment, inputs="text", outputs=["text", "json", "plot"])

# Launch Gradio app to expose the interface as an API
iface.launch(share=True,debug=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e9fc4feb4d2fdb5880.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
